In [1]:
!pip install unidecode
!pip install goose3



  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=1ad9a1c350d4eb90c7dbec7c84283cc35a4fbb78d1924b41264a9a8a225ea386
  Stored in directory: c:\users\carla\appdata\local\pip\cache\wheels\ca\38\d8\dfdfe73bec1d12026b30cb7ce8da650f3f0ea2cf155ea018ae
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=cd4ac09ba295d69d3d50784031d651688323b2eb90676eea780f6045afa5d7f0
  Stored in directory: c:\users\carla\appdata\local\pip\cache\wheels\13\c7\b0\79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built jieba langdetect


In [2]:
import json
import os
import posixpath
from urllib.parse import urlparse
import urllib

import requests
import unidecode
from goose3 import Goose
from tqdm import tqdm

url = "https://raw.githubusercontent.com/billywzh717/N24News/main/nytimes_dataset.json" 
r = urllib.request.urlopen(url)

def resolveComponents(url):
    """
     resolveComponents('http://www.example.com/foo/bar/../../baz/bux/')
    'http://www.example.com/baz/bux/'
     resolveComponents('http://www.example.com/some/path/../file.ext')
    'http://www.example.com/some/file.ext'
    """

    parsed = urlparse(url)
    new_path = posixpath.normpath(parsed.path)
    if parsed.path.endswith('/'):
        # Compensate for issue1707768
        new_path += '/'
    cleaned = parsed._replace(path=new_path)

    return cleaned.geturl()


def clean_article(item, article):
    def clean_text(text):
        text = text.strip()
        text = text.replace('“', '"').replace('”', '"')
        text = text.replace("‘", "'").replace("’", "'")
        return text

    abstract = item['abstract']
    caption = item['caption']
    article = clean_text(article)

    tips = 'As a subscriber, you have 10 gift articles to give each month. Anyone can read what you share.'
    article_cleaned = article.replace(abstract, '').replace(caption, '').replace(tips, '')
    while True:
        if article_cleaned.startswith('\n'):
            article_cleaned = article_cleaned[1:]
        else:
            break

    return article_cleaned


#def download_image(item):
 #   img_data = requests.get(item['image_url'], stream=True).content
  #  with open(os.path.join("./images/%s.jpg" % (item['image_id'])), 'wb') as f:
   #     f.write(img_data)


def download_article(item):
    url = resolveComponents(item['article_url'])
    extract = g.extract(url=url)
    article = unidecode.unidecode(extract.cleaned_text)
    return article


def del_dir(path):
    for i in os.listdir(path):  # os.listdir(path_data)#返回一个列表，里面是当前目录下面的所有东西的相对路径
        file_data = path + '/' + i  # 当前文件夹的下面的所有东西的绝对路径
        if os.path.isfile(file_data):  # os.path.isfile判断是否为文件,如果是文件,就删除.如果是文件夹.递归给del_file.
            os.remove(file_data)



if __name__ == '__main__':
    g = Goose()
    if os.path.exists('./images'):
        del_dir('./images')
    else:
        os.makedirs('./images')
    j = json.loads(r.read().decode(r.info().get_param('charset') or 'utf-8'))
    new_j = []
    for item in tqdm(j):
        #download_image(item)
        item['body'] = clean_article(item, download_article(item))
        new_j.append(item)
    json.dumps(new_j, open(url, 'w', encoding='utf8'))

100%|██████████| 61218/61218 [21:46:52<00:00,  1.28s/it]     


OSError: [Errno 22] Invalid argument: 'https://raw.githubusercontent.com/billywzh717/N24News/main/nytimes_dataset.json'

In [1]:
import pandas as pd

#pd.DataFrame(new_j)


raw_data = pd.DataFrame(new_j, columns=[ "section", "abstract", "article_url", "body", "caption", "headline", "image_id", "image_url"])

NameError: name 'new_j' is not defined

In [5]:
import pandas as pd

#pd.DataFrame(new_j)


raw_data = pd.DataFrame(new_j, columns=[ "section", "abstract", "article_url", "body", "caption", "headline", "image_id", "image_url"])

In [4]:
raw_data.to_csv('C:/Users/carla/Documents/CA683/N24_news_complete_raw_data.csv', index=False)

In [4]:
raw_data = pd.read_csv('C:/Users/carla/Documents/CA683/N24_news_complete_raw_data.csv')

In [6]:
raw_data.dropna()

,section,abstract,article_url,body,caption,headline,image_id,image_url
0,Health,The tobacco and vaping industries and conserva...,https://www.nytimes.com/2019/12/31/health/e-ci...,The Trump administration is expected to announ...,A new study by the National Institute on Drug ...,F.D.A. Plans to Ban Most E-Cigarette Flavors b...,42d25485-0e48-50bf-8d16-948833b2a55d,https://static01.nyt.com/images/2019/11/06/sci...
1,Science,"All year long, Earth passes through streams of...",https://www.nytimes.com/2020/01/01/science/met...,All year long as Earth revolves around the sun...,"Perseid meteors named as ""Orinoid"" streak acro...",Meteor Showers in 2020 That Will Light Up Nigh...,04bc90f0-b20b-511c-b5bb-3ce13194163f,https://static01.nyt.com/images/2020/01/01/sci...
2,Science,A year full of highs and lows in space just en...,https://www.nytimes.com/2020/01/01/science/spa...,"If you follow space news and astronomy, the pa...",Spectators viewing the launch of a Soyuz rocke...,"Rocket Launches, Trips to Mars and More 2020 S...",bd8647b3-8ec6-50aa-95cf-2b81ed12d2dd,https://static01.nyt.com/images/2020/01/01/sci...
3,Television,"""Linda Ronstadt: The Sound of My Voice"" airs o...",https://www.nytimes.com/2020/01/01/arts/televi...,LINDA RONSTADT: THE SOUND OF MY VOICE (2019) 9...,"Linda Ronstadt in ""Linda Ronstadt: The Sound o...",What's on TV Wednesday: A Linda Ronstadt Doc a...,e6c25b53-0416-5795-b0cf-e1243924dc79,https://static01.nyt.com/images/2020/01/01/art...
4,Travel,Interested in the southernmost continent? Here...,https://www.nytimes.com/2020/01/01/travel/anta...,As the number of travelers cruising Antarctica...,"Antarctica21&rsquo;s expedition ship, Ocean No...",New Cruise Ships to Set Sail for Antarctica,98c3d182-95ce-5244-9b9e-008a3dee7354,https://static01.nyt.com/images/2020/01/05/tra...
...,...,...,...,...,...,...,...,...
61213,Global Business,"The Kura ""revolving sushi"" chain has succeeded...",https://www.nytimes.com/2010/12/31/business/gl...,"SAYAMA, Japan The Kura ""revolving sushi"" rest...","Kura, a chain in Japan, relies on small staffs...","For Sushi Chain, Conveyor Belts Carry Profit",a3f8bf67-1938-5d2b-8d83-6e7edb80d5c0,https://static01.nyt.com/images/2010/12/31/bus...
61214,Automobiles,"For the automotive industry, 2010 played out l...",https://www.nytimes.com/2011/01/02/automobiles...,THE last year in automobiles could have been m...,"DEBUTANTES Clockwise from far left, a few news...",Clouds Parted to a Forecast of Sunnier Days,75199d98-0c30-51b9-8efa-4783c078260e,https://static01.nyt.com/images/2011/01/02/aut...
61215,Automobiles,From a sleek European sedan to the last effort...,https://www.nytimes.com/2011/01/02/automobiles...,1. ASTON MARTIN RAPIDE The Rapide is something...,Aston Martin Rapide.,Jerry Garrett's Top 10 Cars of 2010,8af1e9f0-2fb8-57f1-9a10-25ca06743306,https://static01.nyt.com/images/2011/01/02/aut...
61216,Automobiles,From the world's best sports car to an unexpec...,https://www.nytimes.com/2011/01/02/automobiles...,1. FERRARI 458 ITALIA The world's best sports ...,Ferrari 458 Italia.,Lawrence Ulrich's Top 10 Cars of 2010,cfab59b0-2357-54b6-a469-7d5574d60e9c,https://static01.nyt.com/images/2011/01/02/aut...


In [8]:
raw_data['body'].dropna().apply(lambda x: len(x.split(" "))).describe()

count    61200.000000
mean       779.460359
std        515.235040
min          5.000000
25%        268.000000
50%        798.000000
75%       1114.000000
max       7454.000000
Name: body, dtype: float64

In [9]:
raw_data.head()

,section,abstract,article_url,body,caption,headline,image_id,image_url
0,Health,The tobacco and vaping industries and conserva...,https://www.nytimes.com/2019/12/31/health/e-ci...,The Trump administration is expected to announ...,A new study by the National Institute on Drug ...,F.D.A. Plans to Ban Most E-Cigarette Flavors b...,42d25485-0e48-50bf-8d16-948833b2a55d,https://static01.nyt.com/images/2019/11/06/sci...
1,Science,"All year long, Earth passes through streams of...",https://www.nytimes.com/2020/01/01/science/met...,All year long as Earth revolves around the sun...,"Perseid meteors named as ""Orinoid"" streak acro...",Meteor Showers in 2020 That Will Light Up Nigh...,04bc90f0-b20b-511c-b5bb-3ce13194163f,https://static01.nyt.com/images/2020/01/01/sci...
2,Science,A year full of highs and lows in space just en...,https://www.nytimes.com/2020/01/01/science/spa...,"If you follow space news and astronomy, the pa...",Spectators viewing the launch of a Soyuz rocke...,"Rocket Launches, Trips to Mars and More 2020 S...",bd8647b3-8ec6-50aa-95cf-2b81ed12d2dd,https://static01.nyt.com/images/2020/01/01/sci...
3,Television,"""Linda Ronstadt: The Sound of My Voice"" airs o...",https://www.nytimes.com/2020/01/01/arts/televi...,LINDA RONSTADT: THE SOUND OF MY VOICE (2019) 9...,"Linda Ronstadt in ""Linda Ronstadt: The Sound o...",What's on TV Wednesday: A Linda Ronstadt Doc a...,e6c25b53-0416-5795-b0cf-e1243924dc79,https://static01.nyt.com/images/2020/01/01/art...
4,Travel,Interested in the southernmost continent? Here...,https://www.nytimes.com/2020/01/01/travel/anta...,As the number of travelers cruising Antarctica...,"Antarctica21&rsquo;s expedition ship, Ocean No...",New Cruise Ships to Set Sail for Antarctica,98c3d182-95ce-5244-9b9e-008a3dee7354,https://static01.nyt.com/images/2020/01/05/tra...


In [ ]:
print(raw_data["body"][0])

The Trump administration is expected to announce this week that it will ban mint-, fruit- and dessert-flavored e-cigarette cartridges popular with teenagers, but allow menthol and tobacco flavors to remain on the market. Flavored liquid nicotine used in open tank systems can continue to be sold, according to two administration officials who have been briefed on the plan. It is an important concession to vape shops that have thrived alongside the booming e-cigarette business in recent years. President Trump acknowledged late Tuesday that the ban would be announced "very shortly." But he indicated that it might be short-lived and he didn't say which flavors were involved.

The administration's decision is a partial retreat from a commitment it made in September to quickly devise a ban of all flavors except those that tasted like tobacco. Its plan to exempt menthol appeared to be an effort to dodge a bruising legal battle with the tobacco industry, and also reflected intense lobbying by t